## LTEE-FBA-Jenga.ipynb by Rohan Maddamsetti

See documentation at: 
https://cobrapy.readthedocs.io/en/latest/  

In [1]:
import matplotlib as plt
import cobra.test
import cobra
import pandas as pd
import numpy as np
from scipy import stats
from os import path
import random

In [ ]:
## use Gurobi. Necessary for constructing minimal genomes!
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
cobra_config

## Functions.

In [2]:
def make_dict_of_KO_strains(anc_model, gene_vec=[]):
    deletion_cobra_models = {}
    
    gene_str_list = [str(gene_obj) for gene_obj in anc_model.genes]
    if len(gene_vec): ## then filter for the genes (strings) in gene_vec.
        gene_str_list = [x for x in gene_str_list if x in gene_vec]
        
    for genestr in gene_str_list:
        cur_KO_model = anc_model.copy()
        cur_KO_model.id = '_'.join([anc_model.id, genestr, "knockout"])
        cur_gene = cur_KO_model.genes.get_by_id(genestr)
        cobra.manipulation.delete_model_genes(cur_KO_model, cur_gene)
        deletion_cobra_models[genestr] = cur_KO_model
    return deletion_cobra_models

In [3]:
def generate_LTEE_clone_cobra_models(KOed_genes_df, basic_model, using_locus_tag=False):
    
    LTEE_cobra_models = {}
    nonmutator_pops = ["Ara-5", "Ara-6", "Ara+1", "Ara+2", "Ara+4", "Ara+5"]
    mutator_pops = ["Ara-1", "Ara-2", "Ara-3", "Ara-4", "Ara+3", "Ara+6"]
    LTEE_pop_vec = nonmutator_pops + mutator_pops
    model_genes = [x.id for x in basic_model.genes]
    
    for LTEE_pop in LTEE_pop_vec:
        cur_KO_model = basic_model.copy()
        cur_KO_model.id = LTEE_pop + "_50K_A_clone"
        is_cur_pop = (KOed_genes_df["Population"] == LTEE_pop)
        
        if using_locus_tag:
            KOed_genes = [x for x in KOed_genes_df[is_cur_pop].locus_tag]
        else:
            KOed_genes = [x for x in KOed_genes_df[is_cur_pop].blattner]
        
        genes_to_remove = [cur_KO_model.genes.get_by_id(x) for x in KOed_genes if x in model_genes]
        if genes_to_remove:
            cobra.manipulation.delete_model_genes(cur_KO_model, genes_to_remove)
        
        LTEE_cobra_models[LTEE_pop] = cur_KO_model
    
    return LTEE_cobra_models

## Question: have any of the BiGG core, superessential, or specialist/generalist enzymes been knocked out in the 50K LTEE A clones?

In [4]:
KOed_50K_metabolic_genes = pd.read_csv("../results/metabolic-enzymes/KOed-metabolic-enzymes-in-LTEE-50K-A-clones.csv")

KOed_BiGG_core = KOed_50K_metabolic_genes[KOed_50K_metabolic_genes['MetabolicClass'] == "BiGG_core"]
KOed_superessential = KOed_50K_metabolic_genes[KOed_50K_metabolic_genes['MetabolicClass'] == "Superessential"]
KOed_specialist = KOed_50K_metabolic_genes[KOed_50K_metabolic_genes['MetabolicClass'] == "Specialist"]
KOed_generalist = KOed_50K_metabolic_genes[KOed_50K_metabolic_genes['MetabolicClass'] == "Generalist"]

In [5]:
KOed_BiGG_core

,Population,Gene,locus_tag,blattner,gene_length,product,start,end,strand,MetabolicClass
0,Ara-1,appB,ECB_00982,b0979,1137,cytochrome bd-II oxidase subunit II,1056387,1057523,1,BiGG_core
1,Ara-1,pykF,ECB_01645,b1676,1413,pyruvate kinase,1732965,1734377,1,BiGG_core
2,Ara-1,gltB,ECB_03077,b3212,4554,glutamate synthase large subunit,3289867,3294420,1,BiGG_core
3,Ara-1,pflC,ECB_03837,b3952,879,pyruvate formate lyase II activase,4126365,4127243,1,BiGG_core
4,Ara-2,pfkB,ECB_01692,b1723,930,6-phosphofructokinase II,1783635,1784564,1,BiGG_core
5,Ara-2,fbaB,ECB_02025,b2097,1053,fructose-bisphosphate aldolase,2131641,2132693,-1,BiGG_core
6,Ara-2,dld,ECB_02063,b2133,1716,D-lactate dehydrogenase FAD-binding NADH indep...,2173423,2175138,1,BiGG_core
7,Ara-2,eutI,ECB_02349,b2458,1017,predicted phosphotransacetylase subunit,2493318,2494334,-1,BiGG_core
8,Ara-3,kgtP,ECB_02481,b2587,1299,alpha-ketoglutarate transporter,2645941,2647239,-1,BiGG_core
9,Ara-3,tdcE,ECB_02981,b3114,2295,pyruvate formate-lyase 4/2-ketobutyrate format...,3193158,3195452,-1,BiGG_core


In [6]:
KOed_superessential

,Population,Gene,locus_tag,blattner,gene_length,product,start,end,strand,MetabolicClass
41,Ara-1,pgpB,ECB_01255,b1278,765,phosphatidylglycerophosphatase B,1337702,1338466,1,Superessential
42,Ara-2,mrcB,ECB_00148,b0149,2526,penicillin-binding protein 1b,167581,170106,1,Superessential
43,Ara-2,thiD,ECB_02031,b2103,801,phosphomethylpyrimidine kinase,2137845,2138645,-1,Superessential
44,Ara-4,thiD,ECB_02031,b2103,801,phosphomethylpyrimidine kinase,2137845,2138645,-1,Superessential
45,Ara-5,ispF,ECB_02596,b2746,480,2-C-methyl-D-erythritol 24-cyclodiphosphate sy...,2766162,2766641,-1,Superessential
46,Ara-5,ispD,ECB_02597,b2747,711,2-C-methyl-D-erythritol 4-phosphate cytidylylt...,2766641,2767351,-1,Superessential
47,Ara+1,mrdA,ECB_00604,b0635,1902,transpeptidase involved in peptidoglycan synth...,648900,650801,-1,Superessential
48,Ara+1,ispF,ECB_02596,b2746,480,2-C-methyl-D-erythritol 24-cyclodiphosphate sy...,2766162,2766641,-1,Superessential
49,Ara+1,ispD,ECB_02597,b2747,711,2-C-methyl-D-erythritol 4-phosphate cytidylylt...,2766641,2767351,-1,Superessential
50,Ara+3,thiG,ECB_03867,b3991,771,thiazole synthase,4171476,4172246,-1,Superessential


## Basic metabolic models to play with. Default objective is to maximize biomass.

In [7]:
BiGG_model_dir = "../data/BiGG-models"

# the E. coli K-12 iJO1366 model: this is the best curated complete E. coli model.
K12_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "iJO1366.json"))
K12_model.id = "K12"

# the E. coli REL606 iECB_1328 model: most relevant to LTEE, but has stochiometric inconsistencies
## in hydrogen/proton conservation.
REL606_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "iECB_1328.json"))
REL606_model.id = "REL606"

## The simplest well-curated model: E. coli core metabolism.
core_model = cobra.io.load_json_model(path.join(BiGG_model_dir, "e_coli_core.json"))
core_model.id = "Ecoli-core"

In [8]:
## we are simulating media in which glucose import is limiting.
print(K12_model.medium)
print(REL606_model.medium)
print(core_model.medium)

{'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_k_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_cbl1_e': 0.01, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_zn2_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_slnt_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0}
{'EX_ca2_e': 1000.0, 'EX_cbl1_e': 0.01, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_o2_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_zn2_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_slnt_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_k_e': 1000

## Question 1: Measure fitness of 50K A clones using just the E. coli core model.

In [9]:
core_LTEE_models = generate_LTEE_clone_cobra_models(KOed_BiGG_core, core_model)

### Measure the single-KO DFE for each 50K A clone, just using the core metabolic model.

In [10]:
nonmutator_pops = ["Ara-5", "Ara-6", "Ara+1", "Ara+2", "Ara+4", "Ara+5"]
mutator_pops = ["Ara-1", "Ara-2", "Ara-3", "Ara-4", "Ara+3", "Ara+6"]
LTEE_pop_vec = nonmutator_pops + mutator_pops

## Question 2: Measure fitness of 50K A clones using the whole genome metabolic model.

In [11]:
all_KOed_genes_in_50K_A_clones = pd.read_csv("../results/metabolic-enzymes/KOed-genes-in-LTEE-50K-A-clones.csv")
all_KOed_genes_in_50K_A_clones

,Population,Gene,locus_tag,blattner,gene_length,product,start,end,strand
0,Ara-1,mutT,ECB_00100,b0099,390,nucleoside triphosphate pyrophosphohydrolase m...,113848,114237,1
1,Ara-1,ybaL,ECB_00429,b0478,1677,predicted transporter with NADP-binding Rossma...,473629,475305,-1
2,Ara-1,ybcR,ECB_00506,b0554,216,predicted phage lysis protein,548867,549082,1
3,Ara-1,ybcS,ECB_00507,b0555,498,predicted lysozyme,549082,549579,1
4,Ara-1,ybcT,ECB_00508,b0556,462,predicted murein endopeptidase,549576,550037,1
...,...,...,...,...,...,...,...,...,...
2290,Ara+6,rbsR,ECB_03639,b3753,993,DNA-binding transcriptional repressor of ribos...,3900034,3901026,1
2291,Ara+6,frvR,ECB_03782,b3897,1749,predicted regulator,4066254,4068002,-1
2292,Ara+6,metH,ECB_03891,b4019,3684,B12-dependent methionine synthase,4202759,4206442,1
2293,Ara+6,sgcQ,ECB_04168,b4303,807,predicted nucleoside triphosphatase,4508249,4509055,-1


In [12]:
all_KOed_genes_in_50K_A_clones[all_KOed_genes_in_50K_A_clones.Population == "Ara+1"]

,Population,Gene,locus_tag,blattner,gene_length,product,start,end,strand
1194,Ara+1,nhaA,ECB_00018,b0019,1167,pH-dependent sodium/proton antiporter,17488,18654,1
1195,Ara+1,dgt,ECB_00159,b0160,1518,deoxyguanosinetriphosphate triphosphohydrolase,182079,183596,1
1196,Ara+1,lacZ,ECB_00298,b0344,3075,beta-D-galactosidase,335787,338861,-1
1197,Ara+1,ybbO,ECB_00444,b0493,810,short chain dehydrogenase,490508,491317,-1
1198,Ara+1,tesA,ECB_00445,b0494,627,multifunctional acyl-CoA thioesterase I and pr...,491307,491933,-1
...,...,...,...,...,...,...,...,...,...
1517,Ara+1,yjcO,ECB_03950,b4078,690,hypothetical protein,4275258,4275947,-1
1518,Ara+1,rpiR,ECB_03961,b4089,891,DNA-binding transcriptional repressor,4290923,4291813,-1
1519,Ara+1,cycA,ECB_04080,b4208,1413,D-alanine/D-serine/glycine transporter,4414863,4416275,1
1520,Ara+1,hsdS,ECB_04214,b4348,1425,specificity determinant for hsdM and hsdR,4558013,4559437,-1


In [13]:
## Generate models from K-12.
iJO1366_LTEE_models = generate_LTEE_clone_cobra_models(all_KOed_genes_in_50K_A_clones, K12_model)
## Generate models from REL606.
iECB_1328_LTEE_models = generate_LTEE_clone_cobra_models(all_KOed_genes_in_50K_A_clones, REL606_model, using_locus_tag=True)

## Question 3: Examine knockouts of BiGG core genes and superessential genes in the 50K genomes.
## Is there variation in the DFE across LTEE populations?

In [14]:
##BiGG_core_genes = pd.read_csv("")
##bigg_core_gene_vec = 

In [15]:
superessential_genes_df = pd.read_csv("../results/metabolic-enzymes/Barve2012-S6-superessential.csv")
superessential_gene_vec = [x for x in superessential_genes_df.blattner]

## Question 4: Generate minimal metabolic networks.

Question: How much variation in how long to reach a minimal genome?

In [20]:
def play_jenga(basic_model, cutoff = 0.01):
    """
    While there exist genes that have a beneficial effect when knocked out:
    - randomly order the genes to remove.
    - generate a KO. If deleterious, then reject the change and continue.
      If the change is beneficial, then accept and update the model.
    - once the entire DFE of possible KOs is deleterious,
    return the evolved model.
    """
    random.seed() ## seed the PRNG with the current system time.
    evolved_model = basic_model.copy()
    evolved_model.id = "jenga_evolved"
    
    steps = 0 ## number of genes KO'ed so far
    while(True):
        cur_biomass = evolved_model.slim_optimize()
        KO_results = cobra.flux_analysis.single_gene_deletion(evolved_model)
        num_genes = len(KO_results.ids)
        print("num_genes", num_genes)
        
        """ From the Methods of Pal et al. (2006) in Nature:
        A gene was classified as having no fitness effect 
        if the biomass production rate of the knockout strain 
        was reduced by less than a given cutoff; different cutoffs
        led to very similar results.
        
        filter potential knockouts based on this criterion.
        Pal et al. used cutoff = 0.01 and cutoff = 0.1."""

        filtered_KO_results = KO_results[KO_results.growth > (cur_biomass - cutoff)]
        num_candidate_genes = len(filtered_KO_results.ids)
        fract_purifying = float(num_genes - num_candidate_genes)/float(num_genes)
        print("num_candidate_genes", num_candidate_genes)
        print("fraction genes under purifying selection: ", fract_purifying)
        if num_candidate_genes: ## there exist genes to delete
            idx_to_delete = random.sample(range(num_candidate_genes),1).pop()
            genes_to_delete = filtered_KO_results.ids.tolist()[idx_to_delete]
            print("deleting: ", genes_to_delete)
            steps += 1
            print("steps <= " + str(steps))
            cobra.manipulation.remove_genes(evolved_model, genes_to_delete)
        else: ## no more genes to delete
            break ## leave the while loop.  
    print("steps <= " + str(steps))
    return evolved_model

In [ ]:
%%time

evolved_model = play_jenga(REL606_model)

num_genes 1329
num_candidate_genes 1072
fraction genes under purifying selection:  0.19337848006019565
deleting:  {'ECB_01632'}
steps <= 1
num_genes 1328
num_candidate_genes 1071
fraction genes under purifying selection:  0.19352409638554216
deleting:  {'ECB_01401'}
steps <= 2
num_genes 1327
num_candidate_genes 1070
fraction genes under purifying selection:  0.19366993217784476
deleting:  {'ECB_02059'}
steps <= 3
num_genes 1326
num_candidate_genes 1069
fraction genes under purifying selection:  0.193815987933635
deleting:  {'ECB_03478'}
steps <= 4
num_genes 1325
num_candidate_genes 1068
fraction genes under purifying selection:  0.1939622641509434
deleting:  {'ECB_02574'}
steps <= 5
num_genes 1324
num_candidate_genes 1067
fraction genes under purifying selection:  0.19410876132930513
deleting:  {'ECB_00556'}
steps <= 6
num_genes 1323
num_candidate_genes 1066
fraction genes under purifying selection:  0.1942554799697657
deleting:  {'ECB_00687'}
steps <= 7
num_genes 1322
num_candidate_ge

In [ ]:
## 1) generate minimal genomes (see Pal et al. 2006 in Nature, referenced by Hosseini et al. 2018 in PNAS).
##    while generating minimal genomes, plot the percentage of essential genes in the network.
##    Also examine the percentage of essential, superessential, and BiGG core, specialist, and generalist enzymes
##    over time.
## 2) show predicted viability of the evolved LTEE metabolic networks over time.

evolved_model.optimize()
len(evolved_model.genes)

In [ ]:
len(REL606_model.genes)

In [ ]:
137-71